In [2]:
import requests, json
from time import sleep
import sys
import pandas as pd

In [3]:
# Secret token
token = open("ACCESS_TOKEN.txt", "r").read()

In [32]:
# number of artist ids already collected
OFFSET = !wc -l artists.txt
OFFSET = int(OFFSET[0].split()[0])

In [34]:
# constant values.
BASE_URL = "https://api.genius.com"
CLIENT_ACCESS_TOKEN = token
QUERY_SIZE=1105

In [35]:
# send request and get response in json format.
def _get(path, params=None, headers=None):

    # generate request URL
    requrl = '/'.join([BASE_URL, path])
    token = "Bearer {}".format(CLIENT_ACCESS_TOKEN)
    if headers:
        headers['Authorization'] = token
    else:
        headers = {"Authorization": token}

    response = requests.get(url=requrl, params=params, headers=headers)
    response.raise_for_status()

    return response.json()

In [36]:
# read in list of artist names
with open("artistNames_clean.txt", "r") as f:
    artists = f.readlines()
    
artists = [x.strip() for x in artists]
print(str(len(artists))+" artists total")

1810 artists total


In [37]:
# ensure we're not adding duplicates to file
artist_ids = set()
names = set()
with open("artists.txt", "r") as f:
    for line in f:
        line = line.strip()
        try:
            artist_id,n = line.split(',', 1)
        except:
            print(line)
        artist_ids.add(artist_id)
        names.add(n)

print("done loading artist_ids")

done loading artist_ids


In [38]:
# query only a subset of artists at a time to not overwork server
# increment chunk_num on each run
artists = artists[OFFSET:]
artists_chunk = artists[0 : QUERY_SIZE]


# iterate over artist names
for i, artist in enumerate(artists_chunk):
    sys.stdout.write('\r' + str(i).zfill(4))
    
    if artist in names:
        print(" " + artist + ' already queried, skipping')
        continue

    # get artist_id
    find_id = _get("search", {'q': artist})
        
    artist_id = None
    for hit in find_id["response"]["hits"]:
        if hit["result"]["primary_artist"]:
            artist_id = hit["result"]["primary_artist"]["id"]
            break
            
    # check if we've already queried this artist:
    if artist_id in artist_ids:
        print(" *Artist ID " + str(artist_id) + " already queried, skipping")
        continue
    else:
        artist_ids.add(artist_id)
        # write (artist_name,artist_id) pairs to artists.txt file
        with open("artists.txt", "a") as f:
            f.write(str(artist_id)+","+artist+"\n")

    sleep(.25)

0051 *Artist ID 19976 already queried, skipping
0071 *Artist ID 974 already queried, skipping
0174 *Artist ID 12417 already queried, skipping
0203 *Artist ID 9534 already queried, skipping
0241 *Artist ID 213444 already queried, skipping
0309 *Artist ID 9932 already queried, skipping
0315 *Artist ID 12417 already queried, skipping
0385 *Artist ID 438 already queried, skipping
0399 *Artist ID 72 already queried, skipping
0485 *Artist ID 23796 already queried, skipping
0499 *Artist ID 45 already queried, skipping
0528 *Artist ID 1421 already queried, skipping
0546 *Artist ID 2358 already queried, skipping
0548 *Artist ID 498 already queried, skipping
0597 *Artist ID 234499 already queried, skipping
0605 *Artist ID 7811 already queried, skipping
0695 *Artist ID 108 already queried, skipping
0696